# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [3]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
import datetime

import matplotlib.pyplot as plt

import cv2

/home/kevin/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below

GPU limitation

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

2023-07-31 15:20:29.150470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:20:29.165723: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:20:29.165877: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


1 Physical GPUs, 1 Logical GPUs


2023-07-31 15:20:29.167475: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 15:20:29.167802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:20:29.167941: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:20:29.168050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# Building the Models

## Optimized Model 1

In [7]:
#load data using openCV
def load_images_from_folders(folder_path):
    images = [] #list that contains image 
    first_image = True
    prev_rgb = 0
    for filename in os.listdir(folder_path): #looping through the entire folder path
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path) #found the rgb val

        if first_image == True:
            first_image = False
        else:
            prev_rgb = ㄢ
        
        
        images.append(image)
    
    return np.array(images)



moving_data = load_images_from_folders('./datasets/model1_data/Moving')
still_data = load_images_from_folders('./datasets/model1_data/still')


print(moving_data.shape)

print(still_data.shape)



ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 1 has 1 dimension(s)

### Train

In [16]:
data = DataLoader.from_folder("./datasets/model1_data")
train_data, test_data = data.split(0.9)

print(train_data)

2023-07-31 14:38:30.760749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 14:38:30.801949: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 14:38:30.802172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 14:38:30.818181: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

INFO:tensorflow:Load image with size: 4362, num_label: 2, labels: Moving, still.


In [ ]:
model = image_classifier.create(train_data, use_augmentation=True)

### Evaluate & Export

In [ ]:
loss, accuracy = model.evaluate(test_data)
# model.summary()

In [ ]:
model.export(export_dir='.')

In [ ]:
### QUANTIZE
# method 1: save as savedmodel then load and quantize like model2 did
# method 2: change datatype before saving
'''
def representative_dataset():
  for d in test_data:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)
'''

### Batch predicting multiple vids

In [ ]:
falling_paths = ["./datasets/model1_vids/splitted_for_model2/resized_IMG0480_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0484_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0485_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1614_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1615_falling.mp4",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1616_falling.mp4"]

default_paths = ["./datasets/model1_vids/splitted_for_model2/resized_IMG0480_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0484_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG0485_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1614_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1615_default.MOV",
                 "./datasets/model1_vids/splitted_for_model2/resized_IMG1616_default.MOV",
                 "./datasets/model2_vids/resized_jess_jump.MOV",
                 "./datasets/model2_vids/resized_jess_pickup.MOV",
                 "./datasets/model2_vids/resized_jess_run1.MOV",
                 "./datasets/model2_vids/resized_jess_run2.MOV",
                 "./datasets/model2_vids/resized_jess_run3.MOV",
                 "./datasets/model2_vids/resized_jess_run4.MOV",
                 "./datasets/model2_vids/resized_jess_walk1.MOV",
                 "./datasets/model2_vids/resized_jess_walk2.MOV",
                 "./datasets/model2_vids/resized_jess_walk3.MOV",
                 "./datasets/model2_vids/resized_jess_walk4.MOV"]

In [ ]:
def batchPredict(path):
    cap = cv2.VideoCapture(path)
    probs = []
    while True:
        ret, frame = cap.read() 
        if not ret: break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = np.expand_dims(frame_rgb, axis=0)

        interpreter = tf.lite.Interpreter(model_path="model.tflite")
        interpreter.allocate_tensors()
        output = interpreter.get_output_details()
        input = interpreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']

        interpreter.set_tensor(input_index, frame_rgb)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_index)
        output_data = output_data[0]
        output_probs = tf.nn.softmax(output_data.astype(float))

        predicted_index = np.argmax(output_data)
        class_labels = ["Moving", "Still"]
        predicted_class = class_labels[predicted_index]        
        prob = np.around(max(output_probs.numpy()), decimals = 2)

        if predicted_class == "Still": probs.append(np.subtract(1, prob))
        else: probs.append(prob)
    return probs

In [ ]:
def batchPredictMultVids(vid_path_lst):
    record = []
    for vid in vid_path_lst:
        vid_probs = batchPredict(vid)
        print(vid)
        for i in range(len(vid_probs)-15):
            record.append(vid_probs[i:i+16])
            for j in range(i, i+16):
                print(vid_probs[j], end=" ")
            print()
        print()
    return record

In [ ]:
falling = batchPredictMultVids(falling_paths)
default = batchPredictMultVids(default_paths)

## Model 2

### Train

In [8]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

print(falling_data.shape, default_data.shape)

(2344, 16) (6270, 16)


In [7]:

#augment the falling data

mu, sig = 0, 0.001
copy_count = len(default_data)-len(falling_data)
oldLen = len(falling_data)
rowi = 0

# limit = lambda x : max(min(x,1), 0), basically 0 <= x < 1

def limit(x):
    if x > 1: return 1 - (x-1) # if x larger than 1 -> take the difference between x and 1 and subtract from 1
    elif x < 0: return abs(x) # if x is smaller than 0 -> abs val it 
    return x

newLimit = np.vectorize(limit) #Vectorize to loop through the entire array at once

for i in range(copy_count):
    row = falling_data[rowi] + np.random.normal(mu, sig, 16)
    print(row)
    row = newLimit(row)

    falling_data = np.vstack([falling_data, row])
    rowi += 1
    if rowi >= oldLen: rowi = 0

print(falling_data.shape, default_data.shape)

(6270, 16) (6270, 16)


In [15]:
# loading data, output is used to validate input
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data)))) #ones are falling , zeros are default


dataset_size = len(inputs) #length of the entire dataset, including both outputs and inputs
new_indices = np.random.permutation(dataset_size) # using dataset size, shufflen the indices to shuffle X and y at the same time
print(new_indices)

#new_indices is an array that contains all indices of the dataset in a shuffled order, when passed into inputs/outputs, other np arrays, inputs and outputs will be rearranged to match the index of the new_indices array
inputs, outputs = inputs[new_indices], outputs[new_indices] 


train_size = int(0.8*dataset_size)
test_size = dataset_size - train_size


# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train, y_train = inputs[:train_size], outputs[:train_size] #x = images, y = label 
X_test, y_test = inputs[train_size:], outputs[train_size:]


# Define the model architecture
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(100, activation="relu"),
    Dense(100, activation="relu"),
    Dense(10, activation="sigmoid")
])

# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

# Train the model
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=50,
                    validation_data=(X_test, y_test), verbose=0)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Plot the validation accuracy over epochs
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()




[2484 3621 7926 ... 5541 2568 7662]


NameError: name 'Sequential' is not defined

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(x=X_train, y=y_train, epochs=6,callbacks=[tensorboard_callback])

### Evaluate & Export

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
# model.summary()
# model.save("model2")

In [ ]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('model2.tflite', 'wb') as f: f.write(model2_tflite)

In [ ]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)

# Predict

## Reformat Input

In [ ]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [ ]:
# Resize vid
video_path = "compilation_cut.mp4"
output_path = "compilation_cut_resized.mp4"
target_width = 224
target_height = 224

#resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

## Pass into model 1

In [ ]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [ ]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)]
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
print(model2_in)

## Pass into Model 2 and get final prediction

In [ ]:
model2 = tf.keras.models.load_model("model2")

In [ ]:
vid_preds = model2.predict(model2_in)

In [ ]:
threshold = 0.88
bools = vid_preds.reshape((1, len(vid_preds))) > threshold
print(bools)
print(vid_preds.reshape((1, len(vid_preds))))